# Загрузка Pandas и очистка данных

In [1]:
import pandas as pd

In [3]:
data = pd.read_csv('main_task_new.csv')

In [4]:
#Задание модуля три -Проект о вкусной и здоровой пище


data['Number of Reviews'].fillna(0, inplace=True)



df2=pd.get_dummies(data['City'])
data=pd.concat([data,df2], axis=1)

data['Price Range']=data['Price Range'].fillna('unknown')
df3=pd.get_dummies(data['Price Range'])
data=pd.concat([data,df3], axis=1)

#создадим функцию выделяющую дату из отзывов через регулярные выражения
import re
def search_date(i):
    pattern=re.compile('\d{2}\/\d{2}\/\d{4}')
    found_date = pattern.findall(i)    
    if len(found_date) == 0:
        found_date = 'no_date'
    return found_date

 #применим функцию поиска дат к колонке отзывов
data['reviews_date'] = 0
data['Reviews']=data['Reviews'].fillna('no_reviews')
data['reviews_date'] = data['Reviews'].apply(search_date) 

import datetime
from datetime import datetime, timedelta

len_rev = len(data['reviews_date'])

cs_list = []#создадим переменную для заполнения разницы в датах отзывов

#и заполним ее

for i in range (0, len_rev):
    one_list = data['reviews_date'][i]
    if len(one_list) == 2:
        f_date = pd.to_datetime(one_list[0], format='%m/%d/%Y')
        s_date = pd.to_datetime(one_list[1], format='%m/%d/%Y')
        cs_list.append(f_date - s_date)
    else: cs_list.append(timedelta(0))
 #далее создадим новый столбец в датафрейме и переведем в тип данных подходящий для алгоритма,
#то есть извлечем из формата данных 
data['time_delta']=cs_list
data['time_delta']=data['time_delta'].dt.days

# создадим новую переменную для кухонь, пропуски заполним загадкой
data['cuisine'] = data['Cuisine Style'].fillna("['mystery']")

# переведём строки в списки
data['cuisine'] = data['cuisine'].apply(lambda x: eval(x))
#создадим еще один признак как количество кухонь в каждом ресторане
data['cuisine_qntty']=0
for i in range (0,len(data['cuisine'])):
    data['cuisine_qntty'][i]=len(data['cuisine'][i])


#заполним данные о населении городов с помощью словаря population
population={ 'London': [8173900], 'Paris': [2161900],'Madrid':[3223334],'Barcelona':[1620343],'Berlin':[3644826],
            'Milan':[1352562], 'Rome': [2873000], 'Prague': [1309000],'Lisbon':[504718], 'Vienna':[1897000],
            'Amsterdam':[821752],'Brussels':[179000], 'Hamburg':[1841000], 'Munich':[1472000], 'Lyon':[12772], 
            'Stockholm':[975551], 'Budapest':[1756000], 'Warsaw':[1765000], 'Dublin':[544000],'Copenhagen':[603000],
           'Athens':[664000],'Edinburg':[482000],'Zuruch':[402752],'Oporto':[214350],'Geneva':[198979],'Krakow':[766700],
           'Oslo':[690300], 'Helsinki':[650000],'Bratislava':[430000], 'Luxembourg':[620000],'Ljubljana':[300000]}
#и создадим еще один признак
data['city_population']=0
data=data.set_index('City')
data.update(pd.DataFrame(population,index=['city_population']).T)
data=data.reset_index()


<ipython-input-4-9d20956f5495>:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['cuisine_qntty'][i]=len(data['cuisine'][i])


In [6]:
data = data.drop('City', 1)
data = data.drop('Cuisine Style', 1)
data = data.drop('Price Range', 1)
data = data.drop('Reviews', 1)
data = data.drop('URL_TA', 1)
data = data.drop('ID_TA', 1)
data = data.drop('reviews_date', 1)
data = data.drop('cuisine', 1)

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 42 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Restaurant_id      40000 non-null  object 
 1   Ranking            40000 non-null  float64
 2   Rating             40000 non-null  float64
 3   Number of Reviews  40000 non-null  float64
 4   Amsterdam          40000 non-null  uint8  
 5   Athens             40000 non-null  uint8  
 6   Barcelona          40000 non-null  uint8  
 7   Berlin             40000 non-null  uint8  
 8   Bratislava         40000 non-null  uint8  
 9   Brussels           40000 non-null  uint8  
 10  Budapest           40000 non-null  uint8  
 11  Copenhagen         40000 non-null  uint8  
 12  Dublin             40000 non-null  uint8  
 13  Edinburgh          40000 non-null  uint8  
 14  Geneva             40000 non-null  uint8  
 15  Hamburg            40000 non-null  uint8  
 16  Helsinki           400

# Разбиваем датафрейм на части, необходимые для обучения и тестирования модели

In [9]:
# Х - данные с информацией о ресторанах, у - целевая переменная (рейтинги ресторанов)
X = data.drop(['Restaurant_id', 'Rating'], axis = 1)
y = data['Rating']

In [10]:
# Загружаем специальный инструмент для разбивки:
from sklearn.model_selection import train_test_split

In [11]:
# Наборы данных с меткой "train" будут использоваться для обучения модели, "test" - для тестирования.
# Для тестирования мы будем использовать 25% от исходного датасета.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

# Создаём, обучаем и тестируем модель

In [12]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [13]:
# Создаём модель
regr = RandomForestRegressor(n_estimators=100)

# Обучаем модель на тестовом наборе данных
regr.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = regr.predict(X_test)

In [14]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

MAE: 0.2092375
